In [2]:
import numpy as np
from scipy.stats import norm
from scipy.stats import truncnorm
import matplotlib.pyplot as plt
import statistics
import seaborn as sns
import random

import arch 

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import MultiTaskLasso
from sklearn.linear_model import MultiTaskLassoCV

### **III)** Implementation of a method based on Lasso regression in the case of a high degree polynomial (large set if control variates) and comparison with the naive approach *(Question 3)*

**When the number of control variates become too large, the linear regression approach of Step 2 may become too expensive. Let's explain why.**

First of all, when we consider <polynomials, increasing by 1 the degree of the polynomial function increases a lot the number of control variates since, in a $d$-dimensionnal space, a polynomial of order $p$ provides $\binom{d+p}{d}-1$ control variates according to the first paper. Therefore, for instance in our example where we are in a 3-dimensional space, we effectively have $\binom{1+3}{1}-1 = \binom{4}{1}-1 = 3$ control variates $z_1, z_2$ and $z_3$ for the first order polynomial case. For the 2 order polynomial case, we would have $z_1, z_2, z_3, z_1*z_2, z_3*z_3, z_1*z_3, z_1^2, z_2^2$ and $z_3^2$ which effectively corresponds to  $\binom{2+3}{2}-1 = \binom{5}{2}-1 = 9$ control variates. In the same way, we would have $\binom{3+3}{3}-1 = \binom{6}{3}-1 = 19$ for the 3rd order polynomial case. 

On the other side, the second paper states that the computation time of the OLS method is of the order $nm^2 + m^3 + nt$, where $n$ is the number of samples and $m$ the number of control variates. Indeed, when doing OLS, we need to compute $(𝑋′𝑋)^{−1}𝑋′𝑌$. In general, the complexity of the matrix product $AB$ of two matrices $A$ and $B$ with dimensions $a \times b$ and $b \times c$, respectively, can be expressed as $O(abc)$. Therefore, 

a) The matrix product $X'X$ has a complexity of $O(p^2n)$.

b) The matrix-vector product $X'Y$ has a complexity of $O(pn)$.

c) The inverse $(X'X)^{-1}$ has a complexity of $O(p^3)$.

Thus, the overall complexity of these operations is $O(np^2 + p^3)$.

As we saw that the number of controle variates increases rapidly when we increase the degree of the polynomial, this can therefore lead to a significant increase in the complexity of the computation when searching for the optimal coefficients by doing an OLS regression.
That is why the second paper puts forward a solution to this problem which consists of using a Lasso regression to reduce the number of control variates used. 

Indeed, in statistics and machine learning, LASSO (least absolute shrinkage and selection operator) is a regression analysis method that performs both variable selection and regularization in order to enhance the prediction accuracy and interpretability of the resulting statistical model.

It is similar to the OLS but with an additional term which corresponds to lambda, an hyperparameter to find, times the norm 1 of the vector of coefficients. Therefore, in a LASSO regression, we search the parameters that satisfies this : $$\displaystyle \min_{\beta \in \mathbb{R}^p} {\frac{1}{N}} |y - X\beta|_2^2 + \lambda |\beta|_1.$$

Indeed, in statistics and machine learning, LASSO (least absolute shrinkage and selection operator) is a regression analysis method that performs both variable selection and regularization in order to enhance the prediction accuracy and interpretability of the resulting statistical model.

It is similar to the OLS but with an additional term which corresponds to lambda, an hyperparameter to find, times the norm 1 of the vector of coefficients. Therefore, in a LASSO regression, we search the parameters that satisfies this : $$\displaystyle \min_{\beta \in \mathbb{R}^p} {\frac{1}{N}} |y - X\beta|_2^2 + \lambda |\beta|_1.$$

In this question, we are focusing on 2nd order polynomials. As we know that, for quadratic polynomials $P(x) = a^T\times x + \frac{1}{2}\times x^TBx$ (formula of the paper), the re-normalized $\tilde{f}$ is now:$\tilde{f}(x) = f(x) - \frac{1}{2}\operatorname{tr}(B) + (a + Bx)^Tz$. Here, if we note $\alpha_1$, $\alpha_2$, $\alpha_3$, $\alpha_{12}$, $\alpha_{13}$, $\alpha_{23}$, $\alpha_{11}$, $\alpha_{22}$ and $\alpha_{33}$ the coefficients of $z_1$, $z_2$, $z_3$,$z_1\times z_2$, $z_1\times z_3$, $z_2\times z_3$, $z_1^2$, $z_2^2$ and $z_3^2$, then we have : $a = [\alpha_1, \alpha_2, \alpha_3]$ and 
$ B = \begin{pmatrix}
\alpha_{11} & \alpha_{12} & \alpha_{13} \\
\alpha_{21} & \alpha_{22} & \alpha_{23} \\
\alpha_{31} & \alpha_{32} & \alpha_{33} \\
\end{pmatrix}$ but with B symmetric so we only have 6 distinct coefficients in this matrix. 

In [ ]:
def simulate_garch(omega, T):
    h0 = omega[0]/(1 - omega[1] - omega[2])
    std_h = [np.sqrt(h0)]
    returns = [norm.rvs(loc = 0, scale = np.sqrt(h0))]
    for i in range(1,T):
        #formula of h_i defined in the GARCH(1,1)
        h_i = omega[0] + omega[1]* returns[i-1]**2 + omega[2]* std_h[i-1]**2
        std_h.append(np.sqrt(h_i))
        #the returns are normally distributed with a lmean of 0 and a variance of h_i
        returns.append(norm.rvs(loc = 0, scale = np.sqrt(h_i)))
    return np.asarray(returns), std_h 

initial_omega = [0.3,0.3,0.3]

returns = simulate_garch(omega = [0.1,0.2,0.7], T = 1000)[0]

iterations = 2000
burn_in = 1000
std_prior = 10
std_proposal = 0.01

# Posterior desity function of the GARCH(1,1) model
def posterior_density(returns, omega, std_prior):
    n = len(returns)
    variances = np.zeros(n)
    variances[0] = np.var(returns)
    for i in range(1, n):
        variances[i] = omega[0] + omega[1] * returns[i-1]**2 + omega[2] * variances[i-1]
    std = np.sqrt(variances)
    log_likelihood = np.sum(norm.logpdf(returns, 0, std))
    log_prior = np.log(prior(omega, std_prior)) 
    log_posterior = log_likelihood + log_prior
    return log_posterior

# Function that returns the previous one, it will be used for the gradient later
def _post(omega):
    return posterior_density(returns, omega, std_prior)

def gradient(f, x):
    
    """
    It computes the gradient of function f at a point x.
    f is a function with many variables.
    x is the vector that that represents the point where the gradient is computed. 
    """
    h = 1e-6
    gradient = np.zeros_like(x)
    for i in range(x.size):
        x_plus_h = x.copy()
        x_plus_h[i] += h
        gradient[i] = (f(x_plus_h) - f(x)) / h
    return gradient